## What's the Use of Quantum Computers?
Presentation for COMMON, March 2021
[Jack Woehr](mailto:jwoehr@softwoehr.com)

In [ ]:
import numpy as np
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover, AmplificationProblem
from qiskit.circuit.classicalfunction import ClassicalFunction
from qiskit.circuit.library import PhaseOracle
import tempfile
import getpass
import pyodbc
import warnings
from IPython.display import Markdown, display

In [ ]:
user = input("Enter user ID: ")
with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=DeprecationWarning)
    cn = pyodbc.connect(driver="IBM i Access ODBC Driver 64-bit", system="common1.iinthecloud.com", 
                        database="*SYSBAS", uid=user, pwd=getpass.getpass("Password:"))
    display(Markdown("**Connected to database**"))

In [ ]:
cursor = cn.cursor()
cursor.execute("SELECT * FROM DIMACS.SAT3_1")
rows = cursor.fetchall()

In [ ]:
input_3sat = '''
c example DIMACS-CNF 3-SAT
p cnf 3 5
'''
for row in rows:
    input_3sat += str(row[0]) + ' ' + str(row[1]) + ' ' + str(row[2]) + ' ' + '0\n'
print(input_3sat)
fp = tempfile.NamedTemporaryFile(mode='w+t', delete=False)
fp.write(input_3sat)
file_name = fp.name
fp.close()

In [ ]:
oracle = PhaseOracle.from_dimacs_file(file_name)
oracle.draw()

In [ ]:
backend = Aer.get_backend('qasm_simulator')
qinstance = QuantumInstance(backend, shots=1024)

In [ ]:
problem = AmplificationProblem(oracle=oracle, is_good_state=oracle.evaluate_bitstring)
grover = Grover(quantum_instance=qinstance)

result = grover.amplify(problem)
print(result)

In [ ]:
plot_histogram(result.circuit_results)